# Regulatory Information Retrieval and Answer Generation (RIRAG)

This notebook solves the following task of the Regulatory Information Retrieval and Answer Generation competition.

_Using the question and the passages retrieved in Subtask 1 (See ObliQA.ipynb notebook), participants must generate a comprehensive, accurate, and coherent answer. This subtask emphasizes the ability to synthesize information from multiple sources and present it in a clear and logical manner, ensuring that the answer fully addresses the compliance and obligation requirements of the query._

The notebook demonstrates how we can leverage _Retrieval Augmented Generation_ and _Large Language Models_ to synthesize the results obtained through the hybrid (lexical and semantic) search to provide an accurate and precise answer to help professionals navigate the regulatory content.

In [1]:
# Copy RePASs repo to validate our results - ONLY RUN ONCE
#!git clone https://github.com/RegNLP/RePASs.git && cd RePASs

In [2]:
# Load passages from disk
ndocs = 40  # Number of regulatory documents to process
passages = defaultdict(str) # List to store all passages extracted from the regulatory documents

# Extract the passages in each document
for i in range(1, ndocs + 1):
    with open(os.path.join("ObliQADataset/StructuredRegulatoryDocuments", f"{i}.json")) as f:
        doc = json.load(f)  # Loads the contents of the JSON file
        for psg in doc:  # Map each passageId to the actual content
            passages[psg["ID"]] = psg["Passage"]

In [3]:
rankings_dict = defaultdict(list) # Maps a question to the relevant passage and its corresponding ranking score

# Load the rankings file in memory
with open('data/rankings_hybrid.trec', 'r') as f:
    # File format: QuestionID Q0 DocumentID Rank Score Method
    for line in f:
        parts = line.strip().split()
        question_id = parts[0]
        document_id = parts[2]
        rank = int(parts[3])
        score = float(parts[4])
        rankings_dict[question_id].append({
            'doc': document_id,
            'score': score
        })

In [4]:
def extract_passages(question_id: str) -> list[str]:
    """
    Extracts the passages content that are relevant for answering the given question.
    Given a valid question id, it returns at least one passage and up to 10 passages
    that surpass a given relevance threshold.
    
    Args:
        question_id: The question id for which we want to extract the relevant passages
        
    Returns:
        List[str]: A list of passages that are relevant for answering the given question
    """
    retrieved_passages = []
    should_stop = False
    
    for i in range(len(rankings_dict[question_id])):
        # If there was a significant difference in relevance between two passages, don't extract more passages
        # If 10 passages have already been extracted, don't extract more
        if should_stop or len(retrieved_passages) == 10:
            break
            
        # If no passage has been extracted, extract at least one
        if len(retrieved_passages) == 0:
            retrieved_passages.append(rankings_dict[question_id][i]["doc"])
            continue
                
        # Check if there is a relevance difference between this and the next passage of more than 10%
        if i < len(rankings_dict[question_id]) - 1 and rankings_dict[question_id][i]["score"] - rankings_dict[question_id][i+1]["score"] > 0.1:
                should_stop = True

        # Don't include passages with low relevance
        if rankings_dict[question_id][i]["score"] < 0.72:
            break

        retrieved_passages.append(rankings_dict[question_id][i]["doc"])
        
    # Extract the plain text
    retrieved_passages = [passages[doc] for doc in retrieved_passages]
    
    return retrieved_passages

In [5]:
def build_prompt(question: str, relevant_passages: list[str], system_prompt: str = None) -> tuple[str, str]:
    """
    Builds the prompt that will be used to synthesize the passages
    
    Args:
        question: A well formed regulatory question
        relevant_passages: A list of relevant passages that should help answer the question
        system_prompt: Optional custom system prompt. If None, uses default regulatory compliance prompt
    
    Returns:
        A tuple with both the system prompt that contains instructions on how to answer and
        the user prompt that contains the actual question and passages
    """
    
    # Default system prompt if none is provided
    default_system_prompt = """You are a regulatory compliance assistant. Provide a **complete**, **coherent**, and
    **correct** response to the given question by synthesizing the information from the provided passages.
    Your answer should **fully integrate all relevant obligations, best practices, and insights**, and directly
    address the question. The passages are presented in order of relevance, so **prioritize the information
    accordingly** and ensure consistency in your response, avoiding any contradictions. Additionally, reference
    **specific regulations and key compliance requirements** outlined in the regulatory content to support your
    answer. **Do not use any extraneous or external knowledge** outside of the provided passages when crafting
    your response.
    """
    
    # Use provided system prompt or fall back to default
    system_prompt = system_prompt if system_prompt is not None else default_system_prompt

    user_prompt = f"Question: {question}\n\nPassages:\n\n"
    for passage in relevant_passages:
        user_prompt += f"{passage}\n\n"
        
    return (system_prompt, user_prompt)

## Azure OpenAI - Standard deployment

First, we use Azure OpenAI standard deployment to synthesize the retrieved passages for each question using `gpt 3.5 turbo`.
Since the API has enabled rate limit for both token and requests per minute, we use a decorator to throttle the function in the client to prevent sending requests that will be blocked.

In [6]:
# Import azure open AI library to use access API-based LLMs. 
from openai import AzureOpenAI

# Import data structures used for throttling implementation for standard deployments in Azure.
from collections import (
    defaultdict,
    deque
)
from threading import Lock
import asyncio

# Import util libraries
from tqdm import tqdm
from dotenv import load_dotenv
import os
import json
import time

In [7]:
# Load environment variables to handle access to the openAI API using secrets
# Variables to be defined:
# QNA_ENDPOINT_URL: Deployment endpoint for inference/chat completion
# QNA_OPENAI_API_KEY: Key to access openAI API
#
load_dotenv()

False

In [9]:
endpoint = os.getenv('QNA_ENDPOINT_URL')
openAIKey = os.getenv('QNA_OPENAI_API_KEY')
llm_model = 'gpt-35-turbo'

if not endpoint:
    raise ValueError("The environment variable QNA_ENDPOINT_URL is not defined.")

if not openAIKey:
    raise ValueError("The environment variable QNA_OPENAI_API_KEY is not defined.")

openAI_client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=openAIKey,
    api_version="2024-05-01-preview"
)

# This class limits the number of times a function can be called in a given time interval. It guarantees that
# the functions are called all the time, but not in the same order as they are called.
# Limits the number of times we can call a function in a given time interval. 
# Guarantees that a function call will eventually happen, but it does not necessarily respect the order in which 
# the function was called
class Throttle:
    def __init__(self, rate_limit, time_window):
        self.rate_limit = rate_limit # Max number of calls allowed in the given interval
        self.time_window = time_window # The time interval
        self.calls = deque() # Stores the function calls so we can track when to remove calls as they become stale
        self.lock = Lock() # Locks to prevent concurrent access to the function
        self.queue = asyncio.Queue() # Stores function calls that need to be awaited before being executed

    def __call__(self, func):
        async def wrapped_func(*args, **kwargs):
            # Reference the global func
            nonlocal func
            # Lock concurrent access
            with self.lock:
                current_time = time.time()
                
                # Remove function calls that are outside of the time window
                while self.calls and self.calls[0] < current_time - self.time_window:
                    self.calls.popleft()

                # If we can make a call without exceeding the rate limit, then we just call it   
                if len(self.calls) < self.rate_limit:
                    self.calls.append(current_time)
                    return await func(*args, **kwargs)
                else:
                    # Otherwise, queue the function call for later
                    await self.queue.put((func, args, kwargs))
                    
                    # Process function calls in the queue
                    while not self.queue.empty():
                        # Dequeue the function call
                        func, args, kwargs = await self.queue.get()
                        current_time = time.time()

                        # Remove function calls that are outside of the time window
                        while self.calls and self.calls[0] < current_time - self.time_window:
                            self.calls.popleft()

                        # If we can make a call without exceeding the rate limit, then we just call it
                        if len(self.calls) < self.rate_limit:
                            self.calls.append(current_time)
                            result = await func(*args, **kwargs)
                            self.queue.task_done()
                            return result
                        else:
                            # Otherwise, wait a few seconds before retrying 
                            await self.queue.put((func, args, kwargs)) # Enqueue the function call again
                            await asyncio.sleep(min(10, self.time_window))
                    
        return wrapped_func

# Allow maximum 60 calls every 70 seconds
@Throttle(rate_limit=60, time_window=70)
async def summarize_answer(question: str, relevant_passages: list[str]) -> str:
    """
    Summarizes the answer based on the provided passages
    
    Args:
        question: A well formed regulatory question
        relevant_passages: A list of relevant passages that should help answer the question
    """

    (system_prompt, user_prompt) = build_prompt(question, relevant_passages)

    # Executes the LLM API call
    completion = openAI_client.chat.completions.create(
        model=llm_model, # we are using gpt-3.5-turbo
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.25, # Controls how deterministic the response is. Higher values result in more creativity. We want more easy-to-reproduce results
        frequency_penalty=0.0,
        presence_penalty=0.0,
        stop=None,
        stream=False,
        max_tokens=800,
    )

    return completion.choices[0].message.content

In [10]:
answers = []

# load the test dataset
with open("ObliQADataset/ObliQA_test.json") as f:
    data = json.load(f)  # Load the JSON file
    
    # For each question:
    for e in tqdm(data):  # tqdm adds a progress bar
        query = e['Question']  # Extract the actual question
        question_id = e["QuestionID"] # Extract the question id
        
        retrieved_passages = extract_passages(question_id)

        answer = await summarize_answer(query, retrieved_passages)

        answers.append({
            "QuestionID": question_id,
            "RetrievedPassages": retrieved_passages,
            "Answer": answer
        })

# Store the results in a json File
with open("data/answers.json", "w") as f:
    json.dump(answers, f, indent=2)        

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2786/2786 [1:56:08<00:00,  2.50s/it]


## Azure OpenAI - Batch deployment

Second, we use Azure OpenAI batch deployment to synthesize the retrieved passages for each question using `gpt-4o-mini`.
We leverage the batch API to send all queries at once. The response is retrieved offline from the Azure open AI portal.

In [11]:
def queue_batch_summarization_job(jobs):
    """
    Create a batch Job in Azure open AI to generate the answers for all questions with a single request
    """
    endpoint = os.getenv('QNA_ENDPOINT_URL')
    openAIKey = os.getenv('QNA_OPENAI_API_KEY')

    if not endpoint:
        raise ValueError("No se ha definido la variable de entorno QNA_ENDPOINT_URL")

    if not openAIKey:
        raise ValueError("No se ha definido la variable de entorno QNA_OPENAI_API_KEY")

    openAI_client = AzureOpenAI(
        azure_endpoint=endpoint,
        api_key=openAIKey,
        api_version="2024-08-01-preview"
    )

    # File that can either be uploaded manually or programatically
    file_name = "data/batch_questions.jsonl"

    # Save file contents using json lines format
    with open(file_name, 'w') as file:
        for job in jobs:
            file.write(json.dumps(job) + '\n')

    # Upload the file programatically
    batch_file = openAI_client.files.create(
      file=open(file_name, "rb"),
      purpose="batch"
    )
    
    # Wait until the file upload is done
    while True:
        file = openAI_client.files.retrieve(batch_file.id)
        if file.status == "processed" or file.status == "error":
            break
        time.sleep(10)
    
    # Trigger the batch job using the uploaded file
    # Result should terminate in less than 24 hours
    batch_job = openAI_client.batches.create(
      input_file_id=batch_file.id,
      endpoint="/v1/chat/completions",
      completion_window="24h"
    )
    
    return batch_job

In [12]:
jobs = []

# Load the test dataset
with open("ObliQADataset/ObliQA_test.json") as f:
    data = json.load(f)  # Load the JSON file
    
    # For each question:
    for e in tqdm(data):  # tqdm adds a progress bar
        query = e['Question']  # Extract the actual question
        question_id = e["QuestionID"] # Extract the question id

        retrieved_passages = extract_passages(question_id)

        (system_prompt, user_prompt) = build_prompt(query, retrieved_passages)
        
        jobs.append({
            "custom_id": question_id,
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "gpt-4o-mini",
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                "temperature": 0.25,
                "frequency_penalty": 0.0,
                "presence_penalty": 0.0,
                "max_tokens": 800,
            }
        })
        
batch_job = queue_batch_summarization_job(jobs)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2786/2786 [00:00<00:00, 94557.58it/s]


In [13]:
# At this point the result has been downloaded offline and uploaded to the data folder
answers = []

with open("data/batch_result.jsonl") as f:
    # Parse each line of the file as a JSON
    results = [json.loads(line) for line in f]
    
    for result in results:
        # For each result, create an entry in answers array to later create the output file
        question_id = result["custom_id"]
        # Since this function is deterministic, we can just call it again
        # Clearly, there's an optimization we can do to avoid calling this twice
        retrieved_passages = extract_passages(question_id) 
        answer = result["response"]["body"]["choices"][0]["message"]["content"]
        
        answers.append({
            "QuestionID": question_id,
            "RetrievedPassages": retrieved_passages,
            "Answer": answer
        })
        
# Save the results as a JSON file
with open("data/answers-4o.json", "w") as f:
    json.dump(answers, f, indent=2)

## Groq API - Regular deployment

Third, we use Groq's API to synthesize the retrieved passages for each question using `llama-3.1-70b-versatile`.
We leverage Groq's high-performance infrastructure to process queries with minimal latency. The implementation includes 
robust error handling with exponential backoff for rate limits, ensuring reliable processing even at scale.

In [14]:
# Standard library imports
import json
import os
import time
import asyncio
import nest_asyncio
from collections import defaultdict
from threading import Lock

# Third-party imports
from tqdm import tqdm
from dotenv import load_dotenv
from groq import Groq
import backoff

# Load environment variables
load_dotenv()

# Initialize Groq client
groq_api_key = os.getenv('GROQ_API_KEY')

if not groq_api_key:
    raise ValueError("The environment variable GROQ_API_KEY is not defined.")

groq_client = Groq(api_key=groq_api_key)

# Load passages from disk
def load_passages():
    """Load all passages from regulatory documents"""
    ndocs = 40
    passages = defaultdict(str)

    for i in range(1, ndocs + 1):
        with open(os.path.join("ObliQADataset/StructuredRegulatoryDocuments", f"{i}.json")) as f:
            doc = json.load(f)
            for psg in doc:
                passages[psg["ID"]] = psg["Passage"]
    return passages

# Load rankings
def load_rankings():
    """Load rankings from the hybrid search results"""
    rankings_dict = defaultdict(list)
    
    with open('data/rankings_hybrid.trec', 'r') as f:
        for line in f:
            parts = line.strip().split()
            question_id = parts[0]
            document_id = parts[2]
            rank = int(parts[3])
            score = float(parts[4])
            rankings_dict[question_id].append({
                'doc': document_id,
                'score': score
            })
    return rankings_dict

def extract_passages(question_id: str, passages: dict, rankings_dict: dict) -> list[str]:
    """
    Extracts the passages content that are relevant for answering the given question.
    """
    retrieved_passages = []
    should_stop = False
    
    for i in range(len(rankings_dict[question_id])):
        if should_stop or len(retrieved_passages) == 10:
            break
            
        if len(retrieved_passages) == 0:
            retrieved_passages.append(rankings_dict[question_id][i]["doc"])
            continue
                
        if i < len(rankings_dict[question_id]) - 1 and rankings_dict[question_id][i]["score"] - rankings_dict[question_id][i+1]["score"] > 0.1:
                should_stop = True

        if rankings_dict[question_id][i]["score"] < 0.72:
            break

        retrieved_passages.append(rankings_dict[question_id][i]["doc"])
        
    retrieved_passages = [passages[doc] for doc in retrieved_passages]
    
    return retrieved_passages

def build_prompt(question: str, relevant_passages: list[str], system_prompt: str = None) -> tuple[str, str]:
    """
    Builds the prompt that will be used to synthesize the passages
    
    Args:
        question: A well formed regulatory question
        relevant_passages: A list of relevant passages that should help answer the question
        system_prompt: Optional custom system prompt. If None, uses default regulatory compliance prompt
    
    Returns:
        A tuple with both the system prompt that contains instructions on how to answer and
        the user prompt that contains the actual question and passages
    """
    
    # Default system prompt if none is provided
    default_system_prompt = """You are a regulatory compliance assistant. Provide a **complete**, **coherent**, and
    **correct** response to the given question by synthesizing the information from the provided passages.
    Your answer should **fully integrate all relevant obligations, best practices, and insights**, and directly
    address the question. The passages are presented in order of relevance, so **prioritize the information
    accordingly** and ensure consistency in your response, avoiding any contradictions. Additionally, reference
    **specific regulations and key compliance requirements** outlined in the regulatory content to support your
    answer. **Do not use any extraneous or external knowledge** outside of the provided passages when crafting
    your response.
    """
    
    # Use provided system prompt or fall back to default
    system_prompt = system_prompt if system_prompt is not None else default_system_prompt

    user_prompt = f"Question: {question}\n\nPassages:\n\n"
    for passage in relevant_passages:
        user_prompt += f"{passage}\n\n"
        
    return (system_prompt, user_prompt)

@backoff.on_exception(
    backoff.expo,
    (Exception),
    max_tries=5,
    giveup=lambda e: "rate limit" not in str(e).lower()
)
async def summarize_answer_groq(question: str, relevant_passages: list[str]) -> str:
    """
    Summarizes the answer based on the provided passages using Groq's API
    """
    custom_prompt = """As a regulatory compliance assistant, analyze the provided passages
    and answer the accompanying question. Synthesize information from all passages, which
    are presented in order of relevance.

    Key Requirements:
    1. Extract and prioritize mandatory requirements ("shall" statements)
    2. Distinguish between required vs recommended practices
    3. Identify specific deadlines and documentation needs
    4. Use ONLY information from provided passages - no external knowledge
    5. Integrate all obligations and requirements to ensure consistency

    Structure your response as follows:
    1. Brief executive summary (2-3 sentences)
    2. Core requirements with citations [Section XX]
    3. Implementation steps and timeline
    4. Documentation requirements
       - Required reports
       - Retention periods
    5. Special considerations
       - Exemptions
       - Jurisdictional variations
       - Implementation challenges
       - Potential contradictions or conflicts

    Guidelines:
    - Keep responses under 500 words unless complexity requires more
    - Use bullet points for clarity
    - Highlight critical deadlines
    - Flag any ambiguities or conflicts between requirements
    - Process passages in order of presentation
    - Ensure full integration of all relevant requirements
    - Maintain consistency across all recommendations

    Target your response for operational-level compliance officers, emphasizing practical implementation."
    """
    (system_prompt, user_prompt) = build_prompt(question, relevant_passages, custom_prompt)
    try:
        completion = groq_client.chat.completions.create(
            model="llama-3.1-70b-versatile",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.25,
            max_tokens=800,
            top_p=1,
            stream=False
        )
        
        return completion.choices[0].message.content
    
    except Exception as e:
        print(f"Error processing question: {e}")
        raise e

async def main():
    # Load necessary data
    print("Loading passages...")
    passages = load_passages()
    print("Loading rankings...")
    rankings_dict = load_rankings()
    
    answers = []

    # Load the test dataset
    print("Processing questions...")
    with open("ObliQADataset/ObliQA_test.json") as f:
        data = json.load(f)
        
        for e in tqdm(data):
            query = e['Question']
            question_id = e["QuestionID"]
            
            retrieved_passages = extract_passages(question_id, passages, rankings_dict)

            answer = await summarize_answer_groq(query, retrieved_passages)

            answers.append({
                "QuestionID": question_id,
                "RetrievedPassages": retrieved_passages,
                "Answer": answer
            })

    # Store the results in a json File
    print("Saving results...")
    with open("data/answers-llama3.1.json", "w") as f:
        json.dump(answers, f, indent=2)

    print("Processing complete!")

nest_asyncio.apply()
await main()

## Results Evaluation

To evaluate and compare the results obtained from our three different processing methods:
1. Standard Azure OpenAI deployment with GPT-3.5 Turbo
2. Batch Azure OpenAI deployment with GPT-4O-Mini
3. Groq deployment with Llama-3.1-70B-Versatile

Run the following scripts using the RePASs virtual environment to evaluate each model's performance. Make sure you have activated the correct environment before running these commands.

In [15]:
## Script to evaluate the results. Results are placed in /RePASs/data/hybrid or /RePASs/data/hybrid-4o
## These scripts must be run using the virtual env in RePASs

#python scripts/evaluate_model.py --input_file ./../data/answers.json --group_method_name hybrid
#python scripts/evaluate_model.py --input_file ./../data/answers-4o.json --group_method_name hybrid-4o
#python scripts/evaluate_model.py --input_file ./../data/answers-llama3.1.json --group_method_name hybrid-llama